In [ ]:
import datetime
import pandas as pd
import pysrt
import geopy.distance

In [ ]:
def import_log(path):
    df = pd.read_csv(path)
    time = pd.to_datetime(df['Date'] + ' ' + df['Time']).to_frame('time')
    gps = df['GPS'].str.split(' ', 1, expand=True).rename(columns={0:'lat', 1:'lon'})
    gps['lat'] = gps['lat'].astype(float)
    gps['lon'] = gps['lon'].astype(float)
    return time.join(gps).join(df.drop(['Date', 'Time', 'GPS'], 1))

In [ ]:
def calc_gps_speed(df, shift=3):
    def calc_speed(row):
        dist = geopy.distance.distance(row.gps_prev, row.gps_next).km
        time = (row.time_next - row.time_prev) / pd.Timedelta(hours=1)
        return dist / time
    
    t = df[['time', 'lat', 'lon']].copy()
    t['gps'] = t.apply(lambda x: (x.lat, x.lon), axis=1)    
    t['gps_prev'] = t.shift(shift)['gps']
    t['gps_next'] = t.shift(-shift)['gps']
    t['time_prev'] = t.shift(shift)['time']
    t['time_next'] = t.shift(-shift)['time']
    t = t.fillna(method='bfill').fillna(method='ffill')
    return t.apply(calc_speed, axis=1)

def calc_log_betaflight(df):
    df['spd'] = (1.852*df['GSpd(kts)']) # knots to km/h
    df['spd_avg'] = calc_gps_speed(df)
    df['sats'] = df['Tmp2(@C)'] % 100
    df['bat'] = df['VFAS(V)']
    df['curr'] = df.rolling(window=4, min_periods=1, center=True, win_type='gaussian')['Curr(A)'].mean(std=2.5)
    df['eff'] = (df['curr']/df['spd_avg'] * 1000)
    df.loc[df['eff']>999, 'eff'] = 0
    df['thr'] = (df['Thr'] / 20.48 + 50).round(0).astype(int) # -1024/1024 to percent
    df.drop(['GSpd(kts)', 'Tmp2(@C)', 'VFAS(V)', 'Curr(A)', 'Thr'], axis=1, inplace=True)
    return df

def format_log_betaflight(df):
    df['spd'] = df['spd'].round(0).astype(int)
    df['spd_avg'] = df['spd_avg'].round(0).astype(int)
    df['bat'] = df['bat'].round(1)
    df['curr'] = df['bat'].round(0).astype(int)
    df['eff'] = df['eff'].round(0).astype(int)
    df['Alt(m)'] = df['Alt(m)'].round(0).astype(int)
    return df[['time', 'lat', 'lon', 'Alt(m)', '0420', 'RSSI(dB)', 'spd', 'spd_avg', 'sats', 'bat', 'curr', 'eff', 'thr']]

In [ ]:
# export_subtitles

def format_time_srt(s):
    hours, remainder = divmod(s, 3600)
    minutes, remainder = divmod(remainder, 60)
    seconds, remainder = divmod(remainder, 1)
    return '{:02}:{:02}:{:02},{:03}'.format(int(hours), int(minutes), int(seconds), int(remainder*1000))

def format_srt(x):
#     + 'SG=' + str(x.SG) + '   '\
    return str(x.name + 1) + '\n' + format_time_srt(x.start) + ' --> ' + format_time_srt(x.end) + '\n'\
    + str(x.alt) + ' m   '\
    + str(x.spd) + ' km/h   '\
    + str(x.curr) + ' A\n\n'

ssa_header = '''[Script Info]
PlayResX: 1280
PlayResY: 720
WrapStyle: 1

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, Alignment, Outline
Style: Default, Segoe UI Symbol,36,&HFFFFFF,5,1
Style: A1, Segoe UI Symbol,36,&HFFFFFF,7,1
Style: A2, Segoe UI Symbol,36,&HFFFFFF,4,1
Style: A3, Segoe UI Symbol,36,&HFFFFFF,1,1

[Events]
Format: Start, End, Style, Text
'''

def format_time_ssa(s):
    hours, remainder = divmod(s, 3600)
    minutes, remainder = divmod(remainder, 60)
    seconds, remainder = divmod(remainder, 1)
    return f'{int(hours):02}:{int(minutes):02}:{int(seconds):02}.{int(remainder*100):02}'

def format_ssa(x):
    s = f'Dialogue: {format_time_ssa(x.start)},{format_time_ssa(x.end)}' + ',A1,{\pos(16,16)}'\
    + f'📡 {x.sats}\\N'\
    + f' {x["RSSI(dB)"]}\\N'\
    + f' {x["0420"]}\\N'\
    + '\n'
    
    s += f'Dialogue: {format_time_ssa(x.start)},{format_time_ssa(x.end)}' + ',A2,{\pos(16,350)}'\
    + f'{x["Alt(m)"]} m\\N'\
    + f'{x.spd_avg} ㎞/h\\N'\
    + f'EFF {x.eff}\\N'\
    + '\n'
    
    s += f'Dialogue: {format_time_ssa(x.start)},{format_time_ssa(x.end)}' + ',A3,{\pos(16,700)}'\
    + f'🔋{x.curr:2d} A\\N'\
    + f'{x.bat} V\\N'\
    + '\n'
    return s

def export_subtitles_file(df, out_file, formatter):
    ext = '.srt' if formatter == format_srt else '.ssa'
    with open(out_file + ext, "w", encoding='utf-8') as f:
        if formatter == format_ssa:
            f.write(ssa_header)
        for index, row in df.iterrows():
            f.write(formatter(row))

def export_subtitles(df, out_file, formatter=format_ssa, skip_log_rows=0, speed_correction=1, shift_sec=0, split_sec=0):
    max_length_sec = 5
    t = df.iloc[skip_log_rows:].reset_index(drop=True)
    t['start'] = (t['time'] - t.iloc[0]['time']).dt.total_seconds() # seconds from start
    t['start'] = t['start'] * speed_correction # recorder speed deviation compensation
    t['start'] = t['start'] + shift_sec
    t['end'] = t.shift(-1)['start']   
    mask = t['end'] - t['start'] > max_length_sec
    t.loc[mask, 'end'] = t.loc[mask, 'start'] + max_length_sec    
    t.dropna(subset=['end'], inplace=True) # drop last row
        
    if split_sec == 0:
        export_subtitles_file(t, out_file, formatter)
    else:
        n = 0
        while t.shape[0] > 0:
            export_subtitles_file(t[t.start < split_sec], out_file + str(n), formatter)
            t = t[t.start >= split_sec]
            t['start'] = t['start'] - split_sec
            t['end'] = t['end'] - split_sec
            n += 1

In [ ]:
log_dir = 'c:/Dropbox/Projects/FPV/_Logs/Taranis/'
work_dir = 'c:/Projects/FPV/_Video/2021-05-20_Titan_antenna_test/'
df = import_log(log_dir + 'Titan-2021-05-20.csv')
df = calc_log_betaflight(df)
df = format_log_betaflight(df)
export_subtitles(df, work_dir + 'RC_000', skip_log_rows=0, shift_sec=31, split_sec=480)

In [ ]:
t = df[['Alt(m)', 'spd_avg', 'eff']].copy()
t.loc[t['eff']==0, 'eff'] = float('NaN')
t.loc[t['eff']>400, 'eff'] = float('NaN')
t['eff'] = t.rolling(window=20, min_periods=1, center=True)['eff'].mean()
t['eff'] = t['eff']/2
t.plot(figsize=(17,12), grid=True)